In [1]:
import json
from datetime import datetime, timedelta
import os

from tqdm import tqdm
from dotenv import load_dotenv

In [2]:
load_dotenv('env')

True

In [3]:
token = os.environ['GIGACHAT_TOKEN']

from gigachat import GigaChat
import gigachat.context
from gigachat.models import Chat, Messages, MessagesRole 

giga = GigaChat(
    base_url="https://gigachat.devices.sberbank.ru/api/v1/",
    credentials=token,
    model = "GigaChat-Max",
    verify_ssl_certs=False,
)

In [4]:
def make_q1(dialog):
    sys_message = """Ты ассистент по юридическим вопросам. Тебе предлагается фрагмент диалога пользователя с юристом. 
Твоя задача извлечь из фрагмента диалога основную информацию. Извлекай информацию в формате JSON.

Спецификация полей:
problem: str - основная проблема пользователя одной фразой с указанием деталей ситуации.
aspects: Array<str> - упомянутые проблемы вытекающие из первой у пользователя (те второстепенные вопросы, которые пользователя тревожат)
keywords: Array<str> - список ключевых слов или фраз максимально характеризующие проблему и ситуацию пользователя
situation_abstraction: str - максимально полное описание ситуации клиента на основе диалога
query: str - поисковый запрос к справочной правовой системе, который надо задать в поиск правовой системы, чтобы найти ответ на ситуацию пользователя
"""
    payload = Chat(
        messages=[
            Messages(
                role=MessagesRole.SYSTEM,
                content=sys_message
            )
        ],
        temperature=1.,
        top_p=0.,
        repetion_penalty = 1.,
        max_tokens=1000,
        profanity_check=False
    )

    user_prompt = f"""Твоя задача извлечь из фрагмента диалога основную информацию согласно структуре указанной в системном промпте. Извлекай информацию в формате JSON.
<dialog>{dialog}</dialog>"""
    
    
    payload.messages.append(Messages(role=MessagesRole.USER, content=user_prompt))
    
    start_time = datetime.now()
    
    response = giga.chat(payload)

    return response.choices[0].message.content

In [5]:
dialog = """Доброе утро. Вопрос касается налогообложения физического лица. По договору дарения я получил жилой дом в 2015г. В 2021г голу купили квартиру в собственность совместно супругой (50/50). Сейчас планируем продать квартиру. Вопрос. Как вообще мне нужно будет платить налог с продажи квартиры? Будет ли считаться мой жилой дом, которую мне подарили как первое жильё? Думали что при продажи квартиры по истечении трёх лет с момента покупки квартиры не нужно будет платить налоги... Прошу ответить."""

print(dialog)

Доброе утро. Вопрос касается налогообложения физического лица. По договору дарения я получил жилой дом в 2015г. В 2021г голу купили квартиру в собственность совместно супругой (50/50). Сейчас планируем продать квартиру. Вопрос. Как вообще мне нужно будет платить налог с продажи квартиры? Будет ли считаться мой жилой дом, которую мне подарили как первое жильё? Думали что при продажи квартиры по истечении трёх лет с момента покупки квартиры не нужно будет платить налоги... Прошу ответить.


In [10]:
res = make_q1(dialog)

In [11]:
print(res)

{
  "problem": "Налогообложение при продаже квартиры",
  "aspects": [
    "Учет жилого дома, полученного в дар, как первого жилья",
    "Освобождение от уплаты налогов после трех лет владения квартирой"
  ],
  "keywords": [
    "налог", 
    "продажа квартиры", 
    "жилой дом", 
    "дарение", 
    "совместная собственность", 
    "освобождение от налога"
  ],
  "situation_abstraction": "Физическое лицо получило в дар жилой дом в 2015 году и в 2021 году купило квартиру в совместную собственность со своей супругой. Теперь они планируют продать эту квартиру и интересуются, каким образом им придется уплачивать налог с ее продажи, а также учитывается ли полученный ранее в дар жилой дом как их первое жилье.",
  "query": "Налоги при продаже квартиры, полученной в совместной собственности, освобождение от налогов через три года владения"
}
